# The notebook shows the steps taken to generate prediction using a trained BERT model for sequence classification task. The reference link is https://github.com/huggingface/transformers

In [26]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import csv
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
TensorDataset)
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
import os
from tqdm import tqdm, trange

import csv
import logging
import os
import random
import sys
import numpy as np
import pandas as pd

In [27]:
# directory where the model is saved
output_dir = "./bert_output/"

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
tokenizer = BertTokenizer.from_pretrained(output_dir, do_lower_case=True, do_basic_tokenize=True)
model = BertForSequenceClassification.from_pretrained(output_dir,num_labels=2)
model.to(device)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file ./bert_output_cd_ep2/vocab.txt
INFO:pytorch_pretrained_bert.modeling:loading archive file ./bert_output_cd_ep2/
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, eleme

In [30]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b


In [31]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids

In [32]:
class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines

In [33]:
class Sst2Processor(DataProcessor):
    """Processor for the SST-2 data set (GLUE version)."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[0]
            #label = line[1]
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None))
        return examples



In [34]:
test_dir='/directory/to/data' #directory where train.tsv, dev.tsv and test.tsv are stored
processor=Sst2Processor()
test_examples = processor.get_test_examples(test_dir)

In [35]:
label_list = processor.get_labels()
print(label_list)

['0', '1']


In [36]:
def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer, output_mode):
    """Loads a data file into a list of `InputBatch`s.
    Note: the max_seq_length should match the seq_length used during traing of the model
    """

    label_map = {label : i for i, label in enumerate(label_list)}
    print(label_map)

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d" % (ex_index, len(examples)))

        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        #if output_mode == "classification":
         #   label_id = label_map[example.label]
        #elif output_mode == "regression":
         #   label_id = float(example.label)
        #else:
           # raise KeyError(output_mode)
        '''
        
        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
        '''
        
        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              ))
    return features


In [37]:
test_features = convert_examples_to_features(test_examples, label_list, 150, tokenizer, 'classification')

INFO:__main__:Writing example 0 of 97320


{'0': 0, '1': 1}


INFO:__main__:Writing example 10000 of 97320
INFO:__main__:Writing example 20000 of 97320
INFO:__main__:Writing example 30000 of 97320
INFO:__main__:Writing example 40000 of 97320
INFO:__main__:Writing example 50000 of 97320
INFO:__main__:Writing example 60000 of 97320
INFO:__main__:Writing example 70000 of 97320
INFO:__main__:Writing example 80000 of 97320
INFO:__main__:Writing example 90000 of 97320


In [38]:
all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)

In [39]:
test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
# Run prediction for full data
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=16)

In [40]:
def model_prediction(model, test_data_input, output_mode):
    '''
    output_mode: 'classification' or 'regression'
    '''
    model.to(device)
    model.eval()

    nb_test_steps = 0
    preds = []
    for input_ids, input_mask, segment_ids in tqdm(test_data_input, desc="running prediction"):
    #put it on CUDA
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)

        with torch.no_grad():
            logits = model(input_ids, segment_ids, input_mask, labels=None)
        nb_test_steps += 1
        if len(preds) == 0:
            preds.append(logits.detach().cpu().numpy())
        else:
            preds[0] = np.append(preds[0], logits.detach().cpu().numpy(), axis=0)

    preds = preds[0] # logits
    if output_mode == "classification":
        preds_final = np.argmax(preds, axis=1)
    elif output_mode == "regression":
        preds_final= np.squeeze(preds)
    
    return preds


In [41]:
def softmax(x):
    """Compute softmax values to obtain probability from logit."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) 

In [ ]:
pred_arr=model_prediction(model,test_dataloader, 'classification')

running prediction:  67%|██████▋   | 4106/6083 [03:14<01:33, 21.05it/s]

In [21]:
pred_prob=[]
for i in pred_arr:
    softmax_val=softmax(i)
    pred_prob.append(softmax_val[1])
